In [158]:
import pandas as pd
import numpy as np



### Import data

In [159]:
def preprocessing(file,sheet):
    df=pd.read_excel(file,sheet_name=sheet,header=3,parse_dates=['Datetime'])
    df['Datetime']=df.Datetime.dt.strftime('%d-%m-%Y %H:%M:%S')
    df['Datetime']=pd.to_datetime(df['Datetime'])
    return(df)

In [160]:
user=preprocessing('Analytics Position Case Study.xlsx','User Gameplay data')

In [161]:
deposit=preprocessing('Analytics Position Case Study.xlsx','Deposit Data')

In [162]:
withdraw=preprocessing('Analytics Position Case Study.xlsx','Withdrawal Data')

# 1. Find Playerwise Loyalty points earned by Players in the following slots:-
    a. 2nd October Slot S1
    b. 16th October Slot S2
    b. 18th October Slot S1
    b. 26th October Slot S2
    
### Final Loyalty Point Formula
Loyalty Point = (0.01 * deposit) + (0.005 * Withdrawal amount) + (0.001 * (maximum of (#deposit - #withdrawal) or 0)) + (0.2 * Number of games played)



In [163]:
uid=user['User ID'].unique()

In [164]:
def loyalty(date,slot):
    if slot=='s1':
        a=user[(user['Datetime'].dt.day==date) & (user['Datetime'].dt.hour<12) ].groupby('User ID').sum()
        b=deposit[(deposit['Datetime'].dt.day==2) & (deposit['Datetime'].dt.hour<12) ].groupby('User Id').sum()
        b.rename(columns={'Amount':'dep_amt'},inplace=True)
        c=withdraw[(withdraw['Datetime'].dt.day==2) & (withdraw['Datetime'].dt.hour<12) ].groupby('User Id').sum()
        c.rename(columns={'Amount':'withdraw_amt'},inplace=True)
        d=deposit.groupby('User Id').count()
        d.rename(columns={'Datetime':'dep_times'},inplace=True)
        d.drop('Amount',axis=1,inplace=True)
        e=withdraw.groupby('User Id').count()
        e.rename(columns={'Datetime':'withdraw_times'},inplace=True)
        e.drop('Amount',axis=1,inplace=True)

    elif slot=='s2':
        a=user[(user['Datetime'].dt.day==date) & (user['Datetime'].dt.hour>11) ].groupby('User ID').sum()
        b=deposit[(deposit['Datetime'].dt.day==2) & (deposit['Datetime'].dt.hour>11) ].groupby('User Id').sum()
        b.rename(columns={'Amount':'dep_amt'},inplace=True)
        c=withdraw[(withdraw['Datetime'].dt.day==2) & (withdraw['Datetime'].dt.hour>11) ].groupby('User Id').sum()
        c.rename(columns={'Amount':'withdraw_amt'},inplace=True)
        d=deposit.groupby('User Id').count()
        d.rename(columns={'Datetime':'dep_times'},inplace=True)
        d.drop('Amount',axis=1,inplace=True)
        e=withdraw.groupby('User Id').count()
        e.rename(columns={'Datetime':'withdraw_times'},inplace=True)
        e.drop('Amount',axis=1,inplace=True)

    
    index=[i for i in range (1000)]
    slot=pd.DataFrame(index=index)
    slot=pd.concat([slot,a,b,c,d,e],axis=1,join='outer')
    slot.fillna(value=0,inplace=True)
    
    points=[]
    for i in range(len(slot)):
        p=round((0.01*slot['dep_amt'][i]) + (0.005*slot['withdraw_amt'][i]) + (0.001*max(slot['dep_times'][i],slot['withdraw_times'][i])) + (0.2*slot['Games Played'][i]),2)    
        points.append(p)
    
    slot['loyalty_points']=points
    
    return(slot)

#### 1. Playerwise loyalty points  for 2nd October Slot S1

In [165]:
oct_2_s1=loyalty(2,'s1')

In [166]:
oct_2_s1.sort_values('loyalty_points',ascending=False).head(10)

,Games Played,dep_amt,withdraw_amt,dep_times,withdraw_times,loyalty_points
634,0.0,0.0,295671.0,8.0,67.0,1478.42
672,0.0,100000.0,60000.0,35.0,5.0,1300.04
566,1.0,35000.0,180041.0,53.0,3.0,1250.46
949,0.0,5500.0,124500.0,64.0,7.0,677.56
446,1.0,55000.0,0.0,22.0,4.0,550.22
173,0.0,52100.0,0.0,54.0,0.0,521.05
467,3.0,50000.0,0.0,28.0,2.0,500.63
740,0.0,42500.0,0.0,91.0,7.0,425.09
710,51.0,35000.0,0.0,3.0,2.0,360.20
587,13.0,35000.0,0.0,89.0,8.0,352.69


#### 2. Playerwise loyalty points for 16th October Slot S2

In [167]:
oct_16_s2=loyalty(16,'s2')

In [168]:
oct_16_s2.sort_values('loyalty_points',ascending=False).head(10)

,Games Played,dep_amt,withdraw_amt,dep_times,withdraw_times,loyalty_points
634,0.0,0.0,393216.0,8.0,67.0,1966.15
566,1.0,110000.0,0.0,53.0,3.0,1100.25
238,10.0,99000.0,0.0,45.0,7.0,992.04
672,0.0,60000.0,0.0,35.0,5.0,600.04
619,0.0,60000.0,0.0,13.0,0.0,600.01
421,31.0,10000.0,96100.0,99.0,84.0,586.80
415,0.0,50000.0,0.0,6.0,6.0,500.01
352,6.0,47000.0,0.0,128.0,8.0,471.33
538,1.0,40000.0,0.0,27.0,1.0,400.23
182,37.0,35000.0,0.0,16.0,4.0,357.42


#### 3. Playerwise loyalty points for 18th October Slot S1

In [169]:
oct_18_s1=loyalty(18,'s1')

In [170]:
oct_18_s1.sort_values('loyalty_points',ascending=False).head(10)

,Games Played,dep_amt,withdraw_amt,dep_times,withdraw_times,loyalty_points
634,0.0,0.0,295671.0,8.0,67.0,1478.42
672,0.0,100000.0,60000.0,35.0,5.0,1300.04
566,4.0,35000.0,180041.0,53.0,3.0,1251.06
949,0.0,5500.0,124500.0,64.0,7.0,677.56
446,1.0,55000.0,0.0,22.0,4.0,550.22
173,1.0,52100.0,0.0,54.0,0.0,521.25
467,6.0,50000.0,0.0,28.0,2.0,501.23
740,0.0,42500.0,0.0,91.0,7.0,425.09
710,36.0,35000.0,0.0,3.0,2.0,357.20
587,14.0,35000.0,0.0,89.0,8.0,352.89


#### 4. Playerwise loyalty points for 26th October Slot S2

In [171]:
oct_26_s2=loyalty(26,'s2')

In [172]:
oct_26_s2.sort_values('loyalty_points',ascending=False).head(10)

,Games Played,dep_amt,withdraw_amt,dep_times,withdraw_times,loyalty_points
634,1.0,0.0,393216.0,8.0,67.0,1966.35
566,5.0,110000.0,0.0,53.0,3.0,1101.05
238,9.0,99000.0,0.0,45.0,7.0,991.84
672,1.0,60000.0,0.0,35.0,5.0,600.24
619,0.0,60000.0,0.0,13.0,0.0,600.01
421,28.0,10000.0,96100.0,99.0,84.0,586.20
415,0.0,50000.0,0.0,6.0,6.0,500.01
352,4.0,47000.0,0.0,128.0,8.0,470.93
538,2.0,40000.0,0.0,27.0,1.0,400.43
182,44.0,35000.0,0.0,16.0,4.0,358.82


# 2. Calculate overall loyalty points earned and rank players on the basis of loyalty points in the month of October. In case of tie, number of games played should be taken as the next criteria for ranking.

### overall loyalty point

In [173]:
a=user.groupby('User ID').sum()
b=deposit.groupby('User Id').sum()
b.rename(columns={'Amount':'dep_amt'},inplace=True)
c=withdraw.groupby('User Id').sum()
c.rename(columns={'Amount':'withdraw_amt'},inplace=True)
d=deposit.groupby('User Id').count()
d.rename(columns={'Datetime':'dep_times'},inplace=True)
d.drop('Amount',axis=1,inplace=True)
e=withdraw.groupby('User Id').count()
e.rename(columns={'Datetime':'withdraw_times'},inplace=True)
e.drop('Amount',axis=1,inplace=True)

index=[i for i in range (1000)]
octo=pd.DataFrame(index=index)
octo=pd.concat([octo,a,b,c,d,e],axis=1,join='outer')
octo.fillna(value=0,inplace=True)

points=[]
for i in range(len(octo)):
    p=round((0.01*octo['dep_amt'][i]) + (0.005*octo['withdraw_amt'][i]) + (0.001*max(octo['dep_times'][i],octo['withdraw_times'][i])) + (0.2*octo['Games Played'][i]),2)    
    points.append(p)
    
octo['loyalty_points']=points

In [174]:
octo['loyalty_points'].nunique()

936

In [175]:
len(octo['loyalty_points'])

1000

In [176]:
octo

,Games Played,dep_amt,withdraw_amt,dep_times,withdraw_times,loyalty_points
0,15,0.0,0.0,0.0,0.0,3.00
1,8,5000.0,0.0,1.0,0.0,51.60
2,97,567000.0,1270215.0,20.0,20.0,12040.50
3,80,40000.0,0.0,4.0,0.0,416.00
4,5,1750.0,0.0,1.0,0.0,18.50
...,...,...,...,...,...,...
995,18,34500.0,0.0,5.0,0.0,348.60
996,474,96400.0,0.0,31.0,0.0,1058.83
997,109,0.0,0.0,0.0,0.0,21.80
998,4,7400.0,0.0,5.0,0.0,74.80


### Apply Ranking

In [177]:
ranking=octo.sort_values(by=['loyalty_points','Games Played'],ascending=False,)

In [178]:
rank=[i for i in range (1,1001)]
ranking['rank']=rank

In [179]:
ranking.head(10)

,Games Played,dep_amt,withdraw_amt,dep_times,withdraw_times,loyalty_points,rank
634,24,515000.0,15737705.0,8.0,67.0,83843.39,1
99,10,1164800.0,2403141.0,47.0,15.0,23665.75,2
672,10,2158700.0,233750.0,35.0,5.0,22757.78,3
212,1,1924981.0,589850.0,26.0,4.0,22199.29,4
740,2,1738490.0,365288.0,91.0,7.0,19211.83,5
566,183,1819175.0,185071.0,53.0,3.0,19153.76,6
714,6,1676300.0,0.0,34.0,0.0,16764.23,7
421,1557,878600.0,1269809.0,99.0,84.0,15446.54,8
369,37,650000.0,1586208.0,13.0,9.0,14438.45,9
30,13,1329000.0,152145.0,51.0,1.0,14053.38,10


# 3. What is the average deposit amount?

#### Average Deposit per day

In [180]:
dep_avg_day=deposit['Amount'].groupby(deposit['Datetime'].dt.date).mean()

In [181]:
dep_avg_day

Datetime
2022-10-01    4896.937397
2022-10-02    4883.574106
2022-10-03    5154.971480
2022-10-04    4990.715232
2022-10-05    5179.601054
2022-10-06    5584.013986
2022-10-07    5881.150659
2022-10-08    6300.759494
2022-10-09    5074.035654
2022-10-10    5861.287395
2022-10-11    5074.368224
2022-10-12    5309.488414
2022-10-13    5476.302013
2022-10-14    6514.825088
2022-10-15    6181.754296
2022-10-16    5576.052536
2022-10-17    4973.805508
2022-10-18    5404.889465
2022-10-19    5420.777778
2022-10-20    5632.583784
2022-10-21    6100.142574
2022-10-22    4783.895437
2022-10-23    4516.275986
2022-10-24    5825.622018
2022-10-25    5769.710018
2022-10-26    6084.159923
2022-10-27    5456.314545
2022-10-28    6240.975045
2022-10-29    5207.507614
2022-10-30    5385.902946
2022-10-31    5727.503831
Name: Amount, dtype: float64

#### Average deposit amount

In [182]:
deposit['Amount'].mean()

5492.185399701801

# 4. What is the average deposit amount per user in a month?

In [183]:
dep_avg_user=deposit['Amount'].groupby(deposit['User Id']).mean()

In [184]:
dep_avg_user.sample(20)

User Id
687    13512.255814
833    21644.000000
839     2000.000000
141     8060.810811
580    10000.000000
963     3256.562500
551     5000.000000
85      2194.351064
152     5000.000000
77      4262.500000
123     3777.666667
530     1588.888889
724      878.787879
443     1000.000000
946     6906.744186
727    17500.000000
418    10000.000000
950     7393.939394
914     2140.000000
247     5000.000000
Name: Amount, dtype: float64

# 5. What is the average number of games played per user?

In [185]:
game_avg_user=octo['Games Played'].mean()

In [186]:
game_avg_user

355.267

In [187]:
ranking

,Games Played,dep_amt,withdraw_amt,dep_times,withdraw_times,loyalty_points,rank
634,24,515000.0,15737705.0,8.0,67.0,83843.39,1
99,10,1164800.0,2403141.0,47.0,15.0,23665.75,2
672,10,2158700.0,233750.0,35.0,5.0,22757.78,3
212,1,1924981.0,589850.0,26.0,4.0,22199.29,4
740,2,1738490.0,365288.0,91.0,7.0,19211.83,5
...,...,...,...,...,...,...,...
643,2,0.0,0.0,0.0,0.0,0.40,996
857,2,0.0,0.0,0.0,0.0,0.40,997
858,2,0.0,0.0,0.0,0.0,0.40,998
993,2,0.0,0.0,0.0,0.0,0.40,999


# Part B - How much bonus should be allocated to leaderboard players?

### After calculating the loyalty points for the whole month find out which 50 players are at the top of the leaderboard. The company has allocated a pool of Rs 50000 to be given away as bonus money to the loyal players.

### Now the company needs to determine how much bonus money should be given to the players.

### Should they base it on the amount of loyalty points? Should it be based on number of games? Or something else? That’s for you to figure out.

#### Suggest a suitable way to divide the allocated money keeping in mind the following points: 1. Only top 50 ranked players are awarded bonus


In [188]:
top_50=ranking[ranking['rank']<=50]

### players with top 50 rankings

In [189]:
top_50

,Games Played,dep_amt,withdraw_amt,dep_times,withdraw_times,loyalty_points,rank
634,24,515000.0,15737705.0,8.0,67.0,83843.39,1
99,10,1164800.0,2403141.0,47.0,15.0,23665.75,2
672,10,2158700.0,233750.0,35.0,5.0,22757.78,3
212,1,1924981.0,589850.0,26.0,4.0,22199.29,4
740,2,1738490.0,365288.0,91.0,7.0,19211.83,5
566,183,1819175.0,185071.0,53.0,3.0,19153.76,6
714,6,1676300.0,0.0,34.0,0.0,16764.23,7
421,1557,878600.0,1269809.0,99.0,84.0,15446.54,8
369,37,650000.0,1586208.0,13.0,9.0,14438.45,9
30,13,1329000.0,152145.0,51.0,1.0,14053.38,10


### Bonus to top 50 player based on their loyalty points


    formula to calculate bonus 
    
    
    
         company allocated pool                   
    _____________________________________   X   ( Total Games played * 0.011 )   X  ( loyalty points * 0.001 )
      50 (only top 50 ranks get rewards)             
    ________________________________________________________________________________________________________________________ 
     
                                    if withdrawal > 0  ( withdrawal amount * 0.00035 )
                                    
                                    

1. This formula is mainly based on 'Games Played', 'Loyalty Points' & 'Withdrawal Amount'
2. To calculate the bonus amount we are giving more weightage to the 'Games Played', so we can increase the time
3. we are giving negative weightage to the amount of withdrawal, the more money you withdraw the less amount of bonus you will get
     
   

In [190]:
k=top_50.index
l=[]
for i in k:
    
    if top_50['withdraw_amt'][i]!=0:
        rs= ((50000/50) * (top_50['Games Played'][i]*0.011) * ((top_50['loyalty_points'][i]*0.001))) / (top_50['withdraw_amt'][i]*0.00035)
    else:
        rs= ((50000/50) * top_50['Games Played'][i]*0.011) * ((top_50['loyalty_points'][i]*0.001))
    
    l.append(rs)
top_50['Bonus']=l

C:\Users\DELL\AppData\Local\Temp\ipykernel_5164\180197827.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_50['Bonus']=l


In [191]:
top_50

,Games Played,dep_amt,withdraw_amt,dep_times,withdraw_times,loyalty_points,rank,Bonus
634,24,515000.0,15737705.0,8.0,67.0,83843.39,1,4.018494
99,10,1164800.0,2403141.0,47.0,15.0,23665.75,2,3.095036
672,10,2158700.0,233750.0,35.0,5.0,22757.78,3,30.598696
212,1,1924981.0,589850.0,26.0,4.0,22199.29,4,1.182829
740,2,1738490.0,365288.0,91.0,7.0,19211.83,5,3.305887
566,183,1819175.0,185071.0,53.0,3.0,19153.76,6,595.239030
714,6,1676300.0,0.0,34.0,0.0,16764.23,7,1106.439180
421,1557,878600.0,1269809.0,99.0,84.0,15446.54,8,595.259131
369,37,650000.0,1586208.0,13.0,9.0,14438.45,9,10.584901
30,13,1329000.0,152145.0,51.0,1.0,14053.38,10,37.739062


In [192]:
top_50['Bonus'].sum()

49820.32231073974

# Part C - Would you say the loyalty point formula is fair or unfair? 
## Can you suggest any way to make the loyalty point formula more robust?

In [193]:
ranking['Games Played'].median()

48.0

#### Players who withdraws more amount than deposite amount and play less than average games

In [194]:
octo[(octo['dep_amt']<octo['withdraw_amt']) & (octo['Games Played']<octo['Games Played'].mean())]

,Games Played,dep_amt,withdraw_amt,dep_times,withdraw_times,loyalty_points
2,97,567000.0,1270215.0,20.0,20.0,12040.50
7,61,0.0,6617.0,0.0,1.0,45.29
18,7,4950.0,13000.0,5.0,6.0,115.91
20,2,10000.0,20000.0,1.0,2.0,200.40
23,197,22779.0,331000.0,26.0,6.0,1922.22
...,...,...,...,...,...,...
949,1,409100.0,434171.0,64.0,7.0,6262.12
967,9,140000.0,148240.0,6.0,13.0,2143.01
971,231,15000.0,42299.0,3.0,6.0,407.70
972,18,68041.0,69501.0,135.0,29.0,1031.65


# Suggestion
### To calculate Loyalty the weightage given to the 'Withdrawal Amount' is in 'Direct Proportion'. But if we can see there are 17% of users are getting more money than the deposit amount and high loyalty points without playing average number of games. In this cases company is paying more money and giving more benefits based on loyalty points without getting more user traffic. To avoid this our loyalty points should be in 'Inversely Proportional' with withdrawal amount.
 
 
 
 
                               (0.01 * deposit) + (0.001 * deposit times) + (0.2 * Number of games played)
        loyalty points   =    ____________________________________________________________________________________
                                    (0.005 * Withdrawal amount)  +  (0.001 * withdrawl times)

In [195]:
re_l=[]
for i in range(len(octo)):
    
    if octo['withdraw_amt'][i]!=0:
        rel=((0.1*octo['dep_amt'][i]) + (0.001*octo['dep_times'][i]) + (0.2*octo['Games Played'][i])) / ((0.005*octo['withdraw_amt'][i]) + (0.001*octo['withdraw_times'][i]))
    else:
        rel= ((0.1*octo['dep_amt'][i]) + (0.001*octo['dep_times'][i]) + (0.2*octo['Games Played'][i]))
    
    re_l.append(rel)
octo['new_loyalty_points']=re_l

### Comparing Loyalty points calculated by given formula and suggested formula

In [204]:
octo[(octo['dep_amt']<octo['withdraw_amt']) & (octo['Games Played']<octo['Games Played'].mean())].sort_values('loyalty_points',ascending=False).head(10)

,Games Played,dep_amt,withdraw_amt,dep_times,withdraw_times,loyalty_points,new_loyalty_points
634,24,515000.0,15737705.0,8.0,67.0,83843.39,0.654540
99,10,1164800.0,2403141.0,47.0,15.0,23665.75,9.694138
369,37,650000.0,1586208.0,13.0,9.0,14438.45,8.196572
162,2,68320.0,2360000.0,7.0,23.0,12483.62,0.579016
415,26,350000.0,1759843.0,6.0,6.0,12304.42,3.978216
786,7,661500.0,1096160.0,27.0,11.0,12097.23,12.069642
2,97,567000.0,1270215.0,20.0,20.0,12040.50,8.930652
678,12,379000.0,1100000.0,22.0,1.0,9292.42,6.891348
200,4,539998.0,593000.0,8.0,5.0,8365.79,18.212653
344,3,311900.0,1024000.0,18.0,10.0,8239.62,6.091906
